In [1]:
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation,Layer,Lambda

forestfires = pd.read_csv("forestfires.csv")


In [2]:
forestfires.drop(["month","day"],axis=1,inplace = True)

forestfires["size_category"].value_counts()
forestfires.isnull().sum()
forestfires.describe()

,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area,dayfri,...,monthdec,monthfeb,monthjan,monthjul,monthjun,monthmar,monthmay,monthnov,monthoct,monthsep
count,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,...,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000,517.000000
mean,90.644681,110.872340,547.940039,9.021663,18.889168,44.288201,4.017602,0.021663,12.847292,0.164410,...,0.017408,0.038685,0.003868,0.061896,0.032882,0.104449,0.003868,0.001934,0.029014,0.332689
std,5.520111,64.046482,248.066192,4.559477,5.806625,16.317469,1.791653,0.295959,63.655818,0.371006,...,0.130913,0.193029,0.062137,0.241199,0.178500,0.306138,0.062137,0.043980,0.168007,0.471632
min,18.700000,1.100000,7.900000,0.000000,2.200000,15.000000,0.400000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,90.200000,68.600000,437.700000,6.500000,15.500000,33.000000,2.700000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,91.600000,108.300000,664.200000,8.400000,19.300000,42.000000,4.000000,0.000000,0.520000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,92.900000,142.400000,713.900000,10.800000,22.800000,53.000000,4.900000,0.000000,6.570000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,96.200000,291.300000,860.600000,56.100000,33.300000,100.000000,9.400000,6.400000,1090.840000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [3]:
forestfires.loc[forestfires["size_category"]=='small','size_category']=0
forestfires.loc[forestfires["size_category"]=='large','size_category']=1
forestfires["size_category"].value_counts()

0    378
1    139
Name: size_category, dtype: int64

In [4]:
def norm_func(i):
     x = (i-i.min())	/	(i.max()	-	i.min())
     return (x)

In [5]:
predictors = forestfires.iloc[:,0:28]
target = forestfires.iloc[:,28]

In [6]:
predictors1 = norm_func(predictors)

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test= train_test_split(predictors1,target, test_size=0.3,stratify = target)

In [8]:
def prep_model(hidden_dim):
    model = Sequential()
    for i in range(1,len(hidden_dim)-1):
        if (i==1):
            model.add(Dense(hidden_dim[i],input_dim=hidden_dim[0],activation="relu"))
        else:
            model.add(Dense(hidden_dim[i],activation="relu"))
    model.add(Dense(hidden_dim[-1],kernel_initializer="normal",activation="sigmoid"))
    model.compile(loss="binary_crossentropy",optimizer = "rmsprop",metrics = ["accuracy"])
    return model  

In [9]:
first_model = prep_model([28,50,40,20,1])
first_model.fit(np.array(np.asarray(x_train).astype(np.int)),
                np.asarray(y_train).astype(np.int),epochs=500)
pred_train = first_model.predict(np.array(x_train))

<ipython-input-9-7e9c15cb10f1>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  first_model.fit(np.array(np.asarray(x_train).astype(np.int)),
<ipython-input-9-7e9c15cb10f1>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprec

Epoch 1/500
12/12 [==============================] - 0s 916us/step - loss: 0.6808 - accuracy: 0.6454
Epoch 2/500
12/12 [==============================] - 0s 1ms/step - loss: 0.6432 - accuracy: 0.7313
Epoch 3/500
12/12 [==============================] - 0s 999us/step - loss: 0.6076 - accuracy: 0.7313
Epoch 4/500
12/12 [==============================] - 0s 1ms/step - loss: 0.5872 - accuracy: 0.7313
Epoch 5/500
12/12 [==============================] - 0s 1ms/step - loss: 0.5770 - accuracy: 0.7313
Epoch 6/500
12/12 [==============================] - 0s 999us/step - loss: 0.5737 - accuracy: 0.7313
Epoch 7/500
12/12 [==============================] - 0s 999us/step - loss: 0.5714 - accuracy: 0.7313
Epoch 8/500
12/12 [==============================] - 0s 999us/step - loss: 0.5699 - accuracy: 0.7313
Epoch 9/500
12/12 [==============================] - 0s 916us/step - loss: 0.5676 - accuracy: 0.7313
Epoch 10/500
12/12 [==============================] - 0s 999us/step - loss: 0.5661 - accuracy: 0.

12/12 [==============================] - 0s 648us/step - loss: 0.4790 - accuracy: 0.7784
Epoch 83/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4781 - accuracy: 0.7645
Epoch 84/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4769 - accuracy: 0.7812
Epoch 85/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4785 - accuracy: 0.7756
Epoch 86/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4774 - accuracy: 0.7756
Epoch 87/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4756 - accuracy: 0.7784
Epoch 88/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4750 - accuracy: 0.7812
Epoch 89/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4783 - accuracy: 0.7756
Epoch 90/500
12/12 [==============================] - 0s 899us/step - loss: 0.4735 - accuracy: 0.7756
Epoch 91/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4791 - accuracy: 0.7756
Epoch 92/

12/12 [==============================] - 0s 898us/step - loss: 0.4641 - accuracy: 0.7618
Epoch 163/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4699 - accuracy: 0.7729
Epoch 164/500
12/12 [==============================] - 0s 552us/step - loss: 0.4649 - accuracy: 0.7756
Epoch 165/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4642 - accuracy: 0.7701
Epoch 166/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4668 - accuracy: 0.7701
Epoch 167/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4655 - accuracy: 0.7756
Epoch 168/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4674 - accuracy: 0.7784
Epoch 169/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4646 - accuracy: 0.7784
Epoch 170/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4657 - accuracy: 0.7756
Epoch 171/500
12/12 [==============================] - 0s 2ms/step - loss: 0.4680 - accuracy: 0.7756


12/12 [==============================] - 0s 833us/step - loss: 0.4685 - accuracy: 0.7701
Epoch 243/500
12/12 [==============================] - 0s 833us/step - loss: 0.4632 - accuracy: 0.7756
Epoch 244/500
12/12 [==============================] - 0s 916us/step - loss: 0.4646 - accuracy: 0.7784
Epoch 245/500
12/12 [==============================] - 0s 602us/step - loss: 0.4627 - accuracy: 0.7756
Epoch 246/500
12/12 [==============================] - 0s 542us/step - loss: 0.4636 - accuracy: 0.7784
Epoch 247/500
12/12 [==============================] - 0s 969us/step - loss: 0.4609 - accuracy: 0.7784
Epoch 248/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4672 - accuracy: 0.7645
Epoch 249/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4623 - accuracy: 0.7756
Epoch 250/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4631 - accuracy: 0.7701
Epoch 251/500
12/12 [==============================] - 0s 986us/step - loss: 0.4651 - accurac

12/12 [==============================] - 0s 1ms/step - loss: 0.4639 - accuracy: 0.7729
Epoch 323/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4626 - accuracy: 0.7701
Epoch 324/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4632 - accuracy: 0.7729
Epoch 325/500
12/12 [==============================] - 0s 833us/step - loss: 0.4615 - accuracy: 0.7756
Epoch 326/500
12/12 [==============================] - 0s 916us/step - loss: 0.4623 - accuracy: 0.7784
Epoch 327/500
12/12 [==============================] - 0s 999us/step - loss: 0.4639 - accuracy: 0.7784
Epoch 328/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4634 - accuracy: 0.7729
Epoch 329/500
12/12 [==============================] - 0s 999us/step - loss: 0.4625 - accuracy: 0.7729
Epoch 330/500
12/12 [==============================] - 0s 999us/step - loss: 0.4624 - accuracy: 0.7784
Epoch 331/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4614 - accuracy: 0

12/12 [==============================] - 0s 999us/step - loss: 0.4625 - accuracy: 0.7756
Epoch 403/500
12/12 [==============================] - 0s 916us/step - loss: 0.4620 - accuracy: 0.7784
Epoch 404/500
12/12 [==============================] - 0s 916us/step - loss: 0.4611 - accuracy: 0.7784
Epoch 405/500
12/12 [==============================] - 0s 916us/step - loss: 0.4629 - accuracy: 0.7756
Epoch 406/500
12/12 [==============================] - 0s 916us/step - loss: 0.4613 - accuracy: 0.7784
Epoch 407/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4611 - accuracy: 0.7784
Epoch 408/500
12/12 [==============================] - 0s 916us/step - loss: 0.4613 - accuracy: 0.7756
Epoch 409/500
12/12 [==============================] - 0s 999us/step - loss: 0.4594 - accuracy: 0.7756
Epoch 410/500
12/12 [==============================] - 0s 999us/step - loss: 0.4622 - accuracy: 0.7729
Epoch 411/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4627 - accur

Epoch 482/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4603 - accuracy: 0.7756
Epoch 483/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4609 - accuracy: 0.7756
Epoch 484/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4613 - accuracy: 0.7756
Epoch 485/500
12/12 [==============================] - 0s 833us/step - loss: 0.4613 - accuracy: 0.7729
Epoch 486/500
12/12 [==============================] - 0s 916us/step - loss: 0.4608 - accuracy: 0.7756
Epoch 487/500
12/12 [==============================] - 0s 833us/step - loss: 0.4609 - accuracy: 0.7756
Epoch 488/500
12/12 [==============================] - 0s 916us/step - loss: 0.4590 - accuracy: 0.7701
Epoch 489/500
12/12 [==============================] - 0s 3ms/step - loss: 0.4611 - accuracy: 0.7756
Epoch 490/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4639 - accuracy: 0.7784
Epoch 491/500
12/12 [==============================] - 0s 1ms/step - loss: 0.4606 -

In [10]:
pred_train = pd.Series([i[0] for i in pred_train])


In [11]:
size = ["small","large"]
pred_train_class = pd.Series(["small"]*361)
pred_train_class[[i>0.5 for i in pred_train]]= "large"


In [12]:
train = pd.concat([x_train,y_train],axis=1)
train["size_category"].value_counts()

0    264
1     97
Name: size_category, dtype: int64

In [13]:
from sklearn.metrics import confusion_matrix
train["original_class"] = "small"
train.loc[train["size_category"]==1,"original_class"] = "large"
train.original_class.value_counts()
confusion_matrix(pred_train_class,train["original_class"])
np.mean(pred_train_class==pd.Series(train["original_class"]).reset_index(drop=True))
pd.crosstab(pred_train_class,pd.Series(train["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
large,16,1
small,81,263


In [14]:
pred_test = first_model.predict(np.array(x_test))

In [15]:
pred_test = pd.Series([i[0] for i in pred_test])

In [16]:
pred_test_class = pd.Series(["small"]*156)
pred_test_class[[i>0.5 for i in pred_test]] = "large"

In [18]:
pred_test = first_model.predict(np.array(x_test))
pred_test = pd.Series([i[0] for i in pred_test])
pred_test_class = pd.Series(["small"]*156)
pred_test_class[[i>0.5 for i in pred_test]] = "large"
test =pd.concat([x_test,y_test],axis=1)
test["original_class"]="small"
test.loc[test["size_category"]==1,"original_class"] = "large"

test["original_class"].value_counts()
np.mean(pred_test_class==pd.Series(test["original_class"]).reset_index(drop=True)) 


0.6987179487179487

In [19]:
confusion_matrix(pred_test_class,test["original_class"])
pd.crosstab(pred_test_class,pd.Series(test["original_class"]).reset_index(drop=True))

original_class,large,small
row_0,,
large,0,5
small,42,109
